# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import importlib

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

**_After running the code cell below_**

You might need to Alt+Tab to go out of the visualization environment.

In [2]:
env = UnityEnvironment(file_name="./Reacher_Linux/Reacher.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Import the Necessary Packages

In [5]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

## Policy implementation
As seen above, the state and action spaces are continuous. Hence, value based methods such as DQN would not be possible in this case. The policy based methods such as REINFORCE, PPO and Actor-Cirtic methods could be used. 
<br>Please refer the policy_agent_reinforce script for Policy class.

In [6]:
import policy_agent_reinforce
import torch.optim as optim

importlib.reload(policy_agent_reinforce)

policy = policy_agent_reinforce.Policy(state_size, action_size, seed=0)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)

### 4. Instantiate Agent and test it

In the next code cell, an Agent is initiated (Please refer dqn_agent.py for agent details). As the weights for the agent are random, the actions would be selected randomly.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

In [7]:
# An untrained agent

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score

for j in range(100):
    action_probs = policy.act(state)   # select an action
    #actions = np.clip(action_probs.detach().numpy(), -1, 1)              # all actions between -1 and 1
    env_info = env.step(action_probs.detach().numpy())[brain_name]       # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break

print("Score: {}".format(score))

Score: 0.0


## Plain REINFORCE implementation

Lets initially try how simple REINFORCE algorithm works for this case.
To implement REINFORCE, we need to collect trajectories.
<br>Here $t_{max}$=100 is taken as an initial value for collecting trajectories. 

## Important note for REINFORCE in continuous action spaces

The REINFORCE algorithm implementation requires the log probability of taking an action in DISCRETE action spaces. However, for application in continuous action spaces the policy has to be modified so that the output layer gives the mean and variance values of selecting an action. 

Then in order to select an action, the agent needs only to pass the most recent state $s_t$ as input to the network, and then use the output mean $\mu$ and $\sigma^2$ to sample from the distribution:
$$a_t \sim \mathcal{N}(\mu,\sigma^2)$$

Thus, here the policy (i.e. Policy_REINFORCE) outputs $\mu$ and $\sigma$ values for every action.

In [10]:
importlib.reload(policy_agent_reinforce)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

policy = policy_agent_reinforce.Policy_REINFORCE().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)

In [11]:
def reinforce(n_episodes=500, t_max=100, deque_len=100, gamma = 1.0, print_every=10):
    
    scores = [] # Scores list
    scores_deque = deque(maxlen=deque_len)
    
    for i_episode in range(1,n_episodes+1):
       
        # Run an episode i.e. an trajectory with t_max time steps in this case
        action_probs, actions, states, rewards = policy_agent_reinforce.collect_trajectory_REINFORCE(env, policy, t_max)
    
        scores.append(sum(rewards))
        scores_deque.append(sum(rewards))

        # Get the total discounted reward
        discounts = [gamma**i for i in range(len(rewards)+1)]
        discounted_rewards = [a*b for a,b in zip(rewards,discounts)]
        total_disc_reward = sum(discounted_rewards)
        
        # Calculate the loss
        policy_loss = []
        for action_prob in action_probs:
            # Based on the log probability of the action and received reward, calculate the cumulative policy loss
            policy_loss.append(-torch.log(action_prob) * total_disc_reward) 

        policy_loss = torch.cat(policy_loss).sum() # Concatenate all the policy loss values
        
        # See the comments below this cell to understand how the gradients are calculated and updated
        optimizer.zero_grad()
        try:
            policy_loss.backward()
            optimizer.step()
        
        except:
            pass
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=30.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break

    return scores

scores = reinforce()

Episode 10	Average Score: 0.00
Episode 20	Average Score: 0.02
Episode 30	Average Score: 0.01
Episode 40	Average Score: 0.01
Episode 50	Average Score: 0.01
Episode 60	Average Score: 0.01
Episode 70	Average Score: 0.01
Episode 80	Average Score: 0.01
Episode 90	Average Score: 0.00
Episode 100	Average Score: 0.00
Episode 110	Average Score: 0.00
Episode 120	Average Score: 0.00
Episode 130	Average Score: 0.00
Episode 140	Average Score: 0.00
Episode 150	Average Score: 0.00
Episode 160	Average Score: 0.00
Episode 170	Average Score: 0.00
Episode 180	Average Score: 0.00
Episode 190	Average Score: 0.00
Episode 200	Average Score: 0.00
Episode 210	Average Score: 0.00
Episode 220	Average Score: 0.00
Episode 230	Average Score: 0.01
Episode 240	Average Score: 0.00
Episode 250	Average Score: 0.00
Episode 260	Average Score: 0.00
Episode 270	Average Score: 0.00
Episode 280	Average Score: 0.00
Episode 290	Average Score: 0.00
Episode 300	Average Score: 0.00
Episode 310	Average Score: 0.00
Episode 320	Avera

It can be seen that after 500 episodes the agent is hardly learning anything.
Hence, we need better methods to train the agent.

## Training REINFORCE with surrgate functions

you have two choices (usually it's useful to divide by the time since we've normalized our rewards and the time of each trajectory is fixed)

1. $\frac{1}{T}\sum^T_t R_{t}^{\rm future}\log(\pi_{\theta'}(a_t|s_t))$
2. $\frac{1}{T}\sum^T_t R_{t}^{\rm future}\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)}$ where $\theta'=\theta$ and make sure that the no_grad is enabled when performing the division

In [12]:
def discounted_future_rewards(rewards, ratio=0.999):
    print(rewards)
    n = rewards.shape[1]
    step = torch.arange(n)[:,None] - torch.arange(n)[None,:]
    ones = torch.ones_like(step)
    zeros = torch.zeros_like(step)
    
    target = torch.where(step >= 0, ones, zeros)
    step = torch.where(step >= 0, step, zeros)    
    discount = target * (ratio ** step)
    discount = discount.to(device)
    
    rewards_discounted = torch.mm(rewards, discount)
    return rewards_discounted

In [ ]:
# same thing as -policy_loss
def surrogate(policy, states, actions, rewards,
              discount = 0.995, beta=0.01):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # discounted cumulative reward
    rewards = torch.tensor(rewards, dtype=torch.float, device=device)
    R_future = discounted_future_rewards(rewards, discount)

    # subtract baseline (= mean of reward)
    R_mean = torch.mean(R_future)
    R_future -= R_mean

    print(actions)
    
    actions = torch.tensor(actions, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards, dtype=torch.float, device=device)
    

    # policy gradient maxmize target
    surrogates = (R_future * torch.log(actions)).mean()
    
    # include a regularization term
    # this steers new_policy towards 0.5
    # which prevents policy to become exactly 0 or 1
    # this helps with exploration
    # add in 1.e-10 to avoid log(0) which gives nan

    return surrogates

Lsur= surrogate(policy, state, action, reward)

print(Lsur)